# Fundamental Quantum Algorithms: QFT and Grover's Search — Deep Dive

This **deep-dive educational notebook** explains two cornerstone quantum algorithms in detail:

- **Quantum Fourier Transform (QFT)** — mathematical derivation, circuit construction, and comparison with the classical Discrete Fourier Transform (DFT/FFT).
- **Grover's Algorithm** — amplitude amplification, rotation-picture analysis, and comparison with classical search.

Each section contains: theory, circuit-level explanations, annotated code (PennyLane), and small runnable examples on a classical simulator (`default.qubit`).


## Notebook structure

1. QFT — definition, properties, circuit construction, inverse, and example on 3 qubits.
2. Grover — classical baseline, quantum oracle & diffusion, rotation analysis, single/multi-target examples, and a small application analogy for Earth Observation.

This notebook assumes familiarity with basic quantum gates (H, CPhase, X, Z, SWAP) and linear algebra (complex vectors, inner products).

In [ ]:
!pip install pennylane -q

In [ ]:
# Imports and setup
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import math
np.random.seed(42)
print('PennyLane version:', qml.__version__)


------------------------------------------------------------------------

## Part 1 — Quantum Fourier Transform (QFT): Deep Dive

### What the QFT does (mathematical definition)

For an n-qubit system ($N = 2^n$) computational basis states), the QFT is the unitary operator $F$ such that for each computational basis state $|j\rangle$:

$F\,|j\rangle = \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} e^{2\pi i j k / N} |k\rangle$

Crucial points:
- The QFT transforms the *amplitudes* of a quantum state in the computational basis into their discrete Fourier transform (DFT).
- Unlike classical FFT, QFT acts directly on quantum amplitudes and can be exponentially more compact as a circuit for certain subroutines (e.g., phase estimation).

### Classical counterpart: DFT and FFT

- The classical DFT computes the vector $y = F x$ where $F_{k,j} = e^{2πi k j / N}$. Naively this takes $O(N^2)$ complex operations.
- The FFT (Cooley–Tukey) reduces this to O(N log N) by exploiting symmetries.
- QFT doesn't directly speed up computing the full classical DFT of a *classical* array — it speeds up unitary transforms on quantum amplitudes and is a subroutine in algorithms like Shor's.

### Circuit intuition

The standard QFT circuit on `n` qubits: for qubit indices `q_{n-1}` (most significant) down to `q_0` (least significant):

1. Apply H to `q_{n-1}`.
2. Apply controlled phase rotations `CR_k` between `q_{n-1}` (target) and less-significant qubits with angles `π/2, π/4, ...`.
3. Recurse on remaining qubits.
4. Finally, apply SWAPs to reverse qubit order (so output basis order matches the conventional Fourier ordering).

Why phases? A controlled phase `CP(θ)` multiplies basis state |...10...⟩ by `e^{iθ}` only when the control is 1 — building up the complex exponentials `e^{2πijk/N}` across the register.


### Implementation notes (what to watch for in code)

- **Wire ordering convention**: Different implementations sometimes treat wire 0 as the most or least significant bit. Here we document and use a consistent ordering: wire 0 = least significant.
- **Precision**: For visualization we print amplitudes with limited decimals; exact comparisons require complex-phase tolerant checks.
- **Swap gates**: Important to reverse the bit order so the output corresponds to mathematical DFT ordering.


In [ ]:
def qft_rotations(n):
    """Recursively apply rotations for QFT on wires [0..n-1].
    Convention: wire 0 is LSB, wire n-1 is MSB. We apply H to MSB then controlled phases
    from less significant wires onto it.
    """
    if n == 0:
        return
    target = n - 1
    # Hadamard creates equal superposition on target qubit
    qml.Hadamard(wires=target)
    # controlled-phase rotations: each less significant qubit k contributes
    # a phase of exp(i * pi / 2^{distance}) when both qubits are 1.
    for k in range(n - 1):
        angle = np.pi / (2 ** (n - k - 1))
        qml.ControlledPhaseShift(angle, wires=[k, target])
    # recurse on remaining qubits
    qft_rotations(n - 1)

def swap_qubits(n):
    for i in range(n // 2):
        qml.SWAP(wires=[i, n - i - 1])

def qft(n_wires):
    qft_rotations(n_wires)
    swap_qubits(n_wires)

def inverse_qft(n_wires):
    # inverse: undo swaps, then reverse rotations with negative angles
    swap_qubits(n_wires)
    def _inv_rot(n):
        if n == 0:
            return
        _inv_rot(n - 1)
        target = n - 1
        for k in range(n - 1):
            angle = -np.pi / (2 ** (n - k - 1))
            qml.ControlledPhaseShift(angle, wires=[k, target])
        qml.Hadamard(wires=target)
    _inv_rot(n_wires)

print('QFT helpers ready')


### 3-qubit demonstration — expectations

- Input: |3⟩ = |011⟩ (decimal 3). The QFT should output equal-magnitude amplitudes across all 8 basis states with phases encoding the Fourier coefficients.
- We will inspect both magnitudes and phases and verify that inverse QFT recovers the original state.


In [ ]:
# Run QFT demo
n_qubits = 3
dev = qml.device('default.qubit', wires=n_qubits)
@qml.qnode(dev)
def qft_demo(basis_state: int):
    bits = [int(x) for x in format(basis_state, f'0{n_qubits}b')]
    qml.BasisState(np.array(bits), wires=range(n_qubits))
    qft(n_qubits)
    return qml.state()

input_state = 3
out = qft_demo(input_state)
print('Input |3> in binary:', format(input_state, f'0{n_qubits}b'))
print('\nOutput amplitudes (index: complex):')
for i, a in enumerate(out):
    print(f'{i:2d}: {a:.4f}, |a|={np.abs(a):.4f}, arg={np.angle(a):.4f}')


### Visualize amplitude magnitudes and phases

The magnitudes should all be `1/√N` (uniform), while phases vary. The pattern of phases is what encodes the Fourier transform of a single basis spike.

In [ ]:
magnitudes = np.abs(out)
phases = np.angle(out)
labels = [f'|{i}>' for i in range(2**n_qubits)]
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,4))
ax1.bar(range(len(magnitudes)), magnitudes)
ax1.set_xticks(range(len(magnitudes))); ax1.set_xticklabels(labels)
ax1.set_title('Amplitude magnitudes (QFT output)')
ax2.bar(range(len(phases)), phases)
ax2.set_xticks(range(len(phases))); ax2.set_xticklabels(labels)
ax2.set_title('Phases (radians)')
plt.tight_layout(); plt.show()


### Inverse QFT sanity check

We apply QFT then its inverse — the combined unitary should be identity on the state, so measuring should return the original basis index with probability 1 (up to numerical precision).


In [ ]:
@qml.qnode(dev)
def qft_inverse_check(basis_state: int):
    bits = [int(x) for x in format(basis_state, f'0{n_qubits}b')]
    qml.BasisState(np.array(bits), wires=range(n_qubits))
    qft(n_qubits)
    inverse_qft(n_qubits)
    return qml.probs(wires=range(n_qubits))

probs_back = qft_inverse_check(5)
print('Probabilities after QFT → inverse QFT (should be concentrated at original index):')
for i,p in enumerate(probs_back):
    print(f'|{i}>: {p:.6f}')


------------------------------------------------------------------------

## Part 2 — Grover's Algorithm: Deep Dive

### Problem statement and classical baseline

Given an unsorted database of \(N\) items and a function (oracle) that tells whether an item is a solution, classical randomized search needs on average \(N/2\) queries to find a solution (worst-case \(N\)).

Grover's algorithm reduces the number of oracle queries to \(O(\sqrt{N/M})\) when there are \(M\) marked solutions (for a single solution \(M=1\), this is \(O(\sqrt{N})\)).

Key idea: treat the state amplitudes as vectors in a two-dimensional subspace spanned by the *uniform superposition of non-solutions* and the *uniform superposition of solutions*. Each Grover iteration performs a rotation in this subspace, increasing the amplitude in the solution direction.


### Mathematical picture (rotation angle)

Let |s⟩ be the uniform superposition over all `N` basis states, and let |ω⟩ be the (normalized) superposition over the `M` marked states. The initial state |s⟩ can be decomposed as:

$$|s\rangle = \sin(\theta) |\omega\rangle + \cos(\theta) |\omega^{\perp}\rangle,$$

where `sin(θ) = sqrt(M/N)`. Each Grover iteration (Oracle + Diffusion) implements a rotation by angle `2θ` in the 2D subspace. After `r` iterations, the probability to measure a marked state is `sin^2((2r+1)θ)`. The optimal `r` maximizes this expression, approximately `r ≈ π/(4θ) - 1/2`.


### Oracle and diffusion at the gate level

- **Oracle**: flips the sign of marked basis states (phase -1). This is often implemented by mapping the marked state to |11...1⟩ (via X gates on bits that are 0 in the target) and applying a multi-controlled Z.
- **Diffusion**: reflection about the mean. Implemented as `H^n · (2|0⟩⟨0| - I) · H^n`, which equals `H^n · Z_{all-controlled} · H^n` up to X gates — in code we implement the common pattern using `H`, `X`, and a multi-controlled Z.

Note: Real hardware often lacks native multi-controlled gates; they must be decomposed into two-qubit gates, which increases depth.


In [ ]:
def apply_x_if_zero(target_bits):
    for i, b in enumerate(target_bits):
        if b == 0:
            qml.PauliX(wires=i)

def undo_x_if_zero(target_bits):
    for i, b in enumerate(target_bits):
        if b == 0:
            qml.PauliX(wires=i)

def oracle_single(target, n_qubits):
    """Mark a single target by phase-flip. Maps target -> |11..1>, applies multi-controlled Z, then undoes mapping."""
    bits = [int(x) for x in format(target, f'0{n_qubits}b')]
    apply_x_if_zero(bits)
    qml.Hadamard(wires=n_qubits-1)
    controls = list(range(n_qubits-1))
    qml.MultiControlledX(wires=controls + [n_qubits-1], control_values='1'*(n_qubits-1))
    qml.Hadamard(wires=n_qubits-1)
    undo_x_if_zero(bits)

def diffusion(n_qubits):
    # Reflection about the mean: H X (multi-controlled Z) X H pattern
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        qml.PauliX(wires=i)
    qml.Hadamard(wires=n_qubits-1)
    controls = list(range(n_qubits-1))
    qml.MultiControlledX(wires=controls + [n_qubits-1], control_values='1'*(n_qubits-1))
    qml.Hadamard(wires=n_qubits-1)
    for i in range(n_qubits):
        qml.PauliX(wires=i)
        qml.Hadamard(wires=i)

print('Grover primitives (oracle + diffusion) defined')


### Single-target example (3 qubits)

We set `N=8`, target = `|5⟩`. The optimal number of iterations for a single target is approximately `r ≈ π/4 * sqrt(N)` (rounded).

We will print the target probability after each iteration and visualize the rotation behavior.

In [ ]:
n_qubits_g = 3
target = 5
dev_g = qml.device('default.qubit', wires=n_qubits_g)

def grover_circuit_probs(target, n_qubits, n_iters):
    # initialize
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    # apply iterations
    for _ in range(n_iters):
        oracle_single(target, n_qubits)
        diffusion(n_qubits)
    return qml.probs(wires=range(n_qubits))

max_iters = 5
evolution = []
for it in range(max_iters+1):
    @qml.qnode(dev_g)
    def run():
        return grover_circuit_probs(target, n_qubits_g, it)
    probs = run()
    evolution.append(probs)
    print(f'Iteration {it:1d}: target prob = {probs[target]:.6f}, most likely = |{int(np.argmax(probs))}> (p={np.max(probs):.6f})')

evolution = np.array(evolution)


### Visualizing amplitude amplification (rotation)

Plot the probability of the target state after each iteration — you will see it grow, reach a maximum near the theoretical optimal iteration, and then decrease (over-rotation).

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(14,8))
axes = axes.flatten()
N = 2**n_qubits_g
for i in range(6):
    ax = axes[i]
    probs = evolution[i]
    colors = ['red' if j==target else 'steelblue' for j in range(N)]
    ax.bar(range(N), probs, color=colors)
    ax.set_title(f'After {i} iter(s)')
    ax.set_xticks(range(N))
    ax.set_ylim(0,1)
plt.tight_layout(); plt.show()


### Multiple-targets and optimal iterations

If there are `M` marked solutions, the angle `θ` satisfies `sin(θ)=√(M/N)` and the optimal iterations scale as `O(√(N/M))`. Below we illustrate a simple multi-target case by calling the single-target oracle for each marked item (educational but not gate-optimal).

In [ ]:
def multi_oracle(targets, n_qubits):
    for t in targets:
        oracle_single(t, n_qubits)

@qml.qnode(dev_g)
def grover_multi_probs(targets, n_qubits, n_iters):
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    for _ in range(n_iters):
        multi_oracle(targets, n_qubits)
        diffusion(n_qubits)
    return qml.probs(wires=range(n_qubits))

targets = [3,5]
optimal_multi = max(1, int(np.pi/4 * np.sqrt((2**n_qubits_g) / len(targets))))
probs_multi = grover_multi_probs(targets, n_qubits_g, optimal_multi)
print('Targets:', targets, 'Optimal iterations (approx):', optimal_multi)
for i,p in enumerate(probs_multi):
    if p>0.01:
        print(f'|{i}>: {p:.4f}' + ('  <-- target' if i in targets else ''))


### Practical considerations and limitations

- **Oracle construction**: The oracle must be efficiently implementable for the algorithm to be useful. For arbitrary classical checks, building the oracle may cost as much as checking all items classically.
- **Data loading**: Encoding classical data into quantum amplitudes (state preparation) can be expensive and may negate the quantum speedup unless handled carefully (amplitude encoding, quantum RAM assumptions, or native quantum data).
- **NISQ devices**: Multi-controlled gates are deep; on near-term hardware errors may dominate, so Grover's practical use is limited until error rates improve or clever error-mitigation/decompositions are used.

### Classical vs Quantum summary

- **Search**: Classical average O(N), Grover O(√N). Quadratic speedup but not exponential.
- **Transformation**: Classical FFT O(N log N) vs QFT on amplitudes: circuit depth O(n^2) for n qubits representing N=2^n amplitudes; useful when operating on quantum states directly.
